In [ ]:
import multiprocessing as mp

import matplotlib.pyplot as plt
import mkl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import amicrt as am

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
# set numpy threads to 1 so we can use multiprocessing
mkl.set_num_threads(1)

In [ ]:
# sample nonlinear response dataset

def orange(N, D, C=None, n_rel=4):
    assert D >= n_rel, 'D must be >= n_rel'
    if C is None:
        C = np.eye(D)
    X = sampleX(N, D, C)

    if n_rel == 0:
        Y = np.random.randint(0, 2, N)
    else:
        Y = (np.exp(np.sum(X[:, :n_rel]**2, axis=-1) - n_rel) > 0.5).astype(int)

    trueEffects = np.zeros(D)
    trueEffects[:n_rel] = 1

    return X, Y, trueEffects

def sampleX(N, D, C):
    return np.random.multivariate_normal(mean=np.zeros(D), cov=C, size=N)

In [ ]:
# create multi-core pool, use number of CPUs available
pool = mp.Pool(6)

In [ ]:
# dimensionality of data
d = 10
# generate data
x, y, te = orange(2000, d, n_rel=4) # First 4 features are relevant, rest are irrelevant
# split into training and test sets
xTr, xTe, yTr, yTe = train_test_split(x, y)

In [ ]:
# Two different options for AMI-CRT
crtOptions = {
    'ami-crt'  : (am.statistics.ModelBasedStatistic, dict(fn=am.utils.monteCarloEntropy), '>', True),
    'ami-hrt'  : (am.statistics.ModelBasedStatistic, dict(fn=am.utils.monteCarloEntropy), '>', False)
}

# AMI-CRT and AMI-HRT

In [ ]:
# initialize results dictionary
results = dict()

# iterate over crt options
for crtOption in crtOptions:
    statClass, statArgs, ordering, refitStatistic = crtOptions[crtOption]

    # create a new AMI-CRT
    crt = am.CRT(am.conditionals.CompleteConditional,
              dict(),
              statClass,
              statArgs,
              ordering=ordering,
              M=50, # number of null models to fit
              refitStatistic=refitStatistic, tqdm=tqdm, pool=pool, conservative=True)
    
    # which features to test
    testInds = np.arange(d)
    
    # dictionary to indicate which features are continuous (False)
    # and which are discrete (True)
    discInd = {j:False for j in testInds}
    
    # initialize AMI-CRT using training data
    crt.initialize(xTr, yTr, is_discrete=discInd, test_inds=testInds)
    
    # get p-values
    pvalues = crt.fit_evaluate(xTr, yTr, xTe, yTe)
    
    # save results
    results[crtOption] = pvalues

# FAST-AMI-CRT

In [ ]:
statClass, statArgs, ordering, refitStatistic = crtOptions['ami-crt']

# create a new Fast-CRT
fast_crt = am.FastCRT(am.conditionals.CompleteConditional,
                   dict(),
                   statClass,
                   statArgs,
                   ordering=ordering,
                   refitStatistic=refitStatistic,
                   tqdm=tqdm,
                   pool=pool, conservative=False)

# boolean array of same dimensionality as data
# indicates which features are continuous (False) and which are discrete (True)
discInd = np.zeros(d).astype(bool)

# initialize AMI-CRT using training data
fast_crt.initialize(xTr, yTr, is_discrete=discInd)
results['fast-ami-crt'] = fast_crt.fit_evaluate(xTr, yTr, xTe, yTe)

In [ ]:
# plot results
pd.DataFrame(results).plot.bar()
plt.ylabel('$p$-value')
plt.xlabel('Feature #')
plt.title('$p$-values for each feature')

In [ ]:
pool.close()